In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow_probability.substrates import numpy as tfp

In [ ]:
!pip install cmdstanpy
!pip install git+https://github.com/OriolAbril/arviz.git@ci

In [2]:
from cmdstanpy import install_cmdstan
install_cmdstan()

CmdStan install directory: C:\Users\39339\.cmdstan
CmdStan version 2.33.1 already installed
Test model compilation


True

In [3]:
from cmdstanpy import CmdStanModel, set_cmdstan_path
import arviz as az

In [41]:
X = pd.read_csv("../Datasets/covariates.csv")

In [42]:
type_mapping = {'rural': 0, 'urban': 1, 'factory': 2}
X['Type'] = X['Type'].map(type_mapping)

In [43]:
Y = pd.read_csv("../Datasets/Dataset_120.csv")

In [44]:
rows_to_drop_Y = Y[Y['Count_120'].isna()].index

# Remove corresponding rows from dataset X
X = X.drop(index=rows_to_drop_Y)
Y = Y.drop(index=rows_to_drop_Y)
stations = X['Station']
years = X['Year']

# If you want to reset the index after dropping rows
X = X.reset_index(drop=True)
Y = Y.reset_index(drop=True)


In [38]:
prior_elic = """

data {
  int<lower=1> N; // Number of observations
  int<lower=1> P; // Covariate number (should be around 10)
  int<lower=1> nyears;
  int<lower=1> nstations;


  array[N] int<lower=1> station;
  array[N] int<lower=1> year;
  // array[N] int<lower=4, upper=10> month;

  array[N] int<lower=0> y; // Count data
  matrix[N, P] X; // Predictor matrix
}

parameters {
  vector[P] beta; // Coefficients for predictors
  vector[nstations] eta; // Random effects for comuni
  vector[nyears] xi; // Random effects for years

  //real<lower=0> sigma0; // Standard deviation for beta
  //real<lower=0> sigma1; // Standard deviation for xi
  //real<lower=0> sigma2; // Standard deviation for eta
}

transformed parameters {
    vector[N] lambda;
    vector[N] intercept;
    vector[N] fix_eff;

    intercept = xi[year] + eta[station];
    fix_eff = X * beta;

    lambda = exp(intercept + fix_eff);
}

model {

  beta ~ normal(0, 10);
  xi ~ normal(0, 10);
  eta ~ normal(0, 10);

  for (n in 1:N) {
    y[n] ~ poisson(lambda[n]);
  }

}

"""

stan_file = "./priors.stan"

with open(stan_file, "w") as f:
    print(prior_elic, file=f)

priors = CmdStanModel(stan_file=stan_file)

12:31:32 - cmdstanpy - INFO - compiling stan file C:\Users\39339\OneDrive\Desktop\ISEMES~1\BAYESI~2\main\Modello\priors.stan to exe file C:\Users\39339\OneDrive\Desktop\I semestre\Bayesian_Project\main\Modello\priors.exe
12:32:33 - cmdstanpy - INFO - compiled model executable: C:\Users\39339\OneDrive\Desktop\I semestre\Bayesian_Project\main\Modello\priors.exe


In [45]:
Y['Count_120'] = pd.to_numeric(Y['Count_120']).astype('Int64')

In [46]:
stations = X['Station']
years = X['Year']

In [47]:
stations = stations.map(dict(zip(unique_values, (np.argsort(stations.unique()) % stations.unique().shape[0]) + 1)))

In [49]:
years = years - 2009

In [51]:
X = X.drop(['Year','Station'], axis  = 1)

In [52]:
data = {
    "N": len(Y),
    "P": X.shape[1],
    "nyears": 13,
    "nstations": 45,
    "station": stations,
    "year": years,
    "y": Y['Count_120'],
    "X": X
}


fit = priors.sample(data=data, chains=4, parallel_chains=4, 
    iter_warmup=1000, iter_sampling=1000)
prior_az = az.from_cmdstanpy(fit)

12:34:56 - cmdstanpy - INFO - CmdStan start processing


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

12:34:56 - cmdstanpy - ERROR - Chain [2] error: error during processing Operation not permitted
12:34:56 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
12:34:56 - cmdstanpy - ERROR - Chain [3] error: error during processing Operation not permitted
12:34:56 - cmdstanpy - ERROR - Chain [4] error: error during processing Operation not permitted


12:34:56 - cmdstanpy - INFO - CmdStan done processing.


RuntimeError: Error during sampling:
Exception: poisson_lpmf: Rate parameter is nan, but must be nonnegative! (in 'priors.stan', line 46, column 4 to column 30)
	Exception: poisson_lpmf: Rate parameter is nan, but must be nonnegative! (in 'priors.stan', line 46, column 4 to column 30)
	Exception: poisson_lpmf: Rate parameter is nan, but must be nonnegative! (in 'priors.stan', line 46, column 4 to column 30)
	Exception: poisson_lpmf: Rate parameter is nan, but must be nonnegative! (in 'priors.stan', line 46, column 4 to column 30)
	Exception: poisson_lpmf: Rate parameter is nan, but must be nonnegative! (in 'priors.stan', line 46, column 4 to column 30)
	Exception: poisson_lpmf: Rate parameter is nan, but must be nonnegative! (in 'priors.stan', line 46, column 4 to column 30)
	Exception: poisson_lpmf: Rate parameter is nan, but must be nonnegative! (in 'priors.stan', line 46, column 4 to column 30)
	Exception: poisson_lpmf: Rate parameter is nan, but must be nonnegative! (in 'priors.sta